# Clustering

Objectif: my task will be to try to cluster clients of a Wholesale Distributor
based off of the sales of some product categories

In [31]:
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.evaluation.ClusteringEvaluator

import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.evaluation.ClusteringEvaluator


In [34]:
val file ="Machine_Learning_Sections/Clustering/Wholesale customers data.csv"
val df = spark.read.option("header","true").option("inferSchema","true").format("csv").load(file)

file: String = Machine_Learning_Sections/Clustering/Wholesale customers data.csv
df: org.apache.spark.sql.DataFrame = [Channel: int, Region: int ... 6 more fields]


In [35]:
df.show(5)

+-------+------+-----+----+-------+------+----------------+----------+
|Channel|Region|Fresh|Milk|Grocery|Frozen|Detergents_Paper|Delicassen|
+-------+------+-----+----+-------+------+----------------+----------+
|      2|     3|12669|9656|   7561|   214|            2674|      1338|
|      2|     3| 7057|9810|   9568|  1762|            3293|      1776|
|      2|     3| 6353|8808|   7684|  2405|            3516|      7844|
|      1|     3|13265|1196|   4221|  6404|             507|      1788|
|      2|     3|22615|5410|   7198|  3915|            1777|      5185|
+-------+------+-----+----+-------+------+----------------+----------+
only showing top 5 rows



In [36]:
val data = df.select($"Fresh",$"Milk",$"Grocery",$"Frozen",$"Detergents_Paper",$"Delicassen")

data: org.apache.spark.sql.DataFrame = [Fresh: int, Milk: int ... 4 more fields]


In [37]:
data.printSchema

root
 |-- Fresh: integer (nullable = true)
 |-- Milk: integer (nullable = true)
 |-- Grocery: integer (nullable = true)
 |-- Frozen: integer (nullable = true)
 |-- Detergents_Paper: integer (nullable = true)
 |-- Delicassen: integer (nullable = true)



In [38]:
// number of row contain null element 
data.filter(row => row.anyNull).count

res14: Long = 0


In [39]:
val assembler = new VectorAssembler().setInputCols(Array("Fresh",
		 "Milk",
		 "Grocery",
		 "Frozen",
		 "Detergents_Paper","Delicassen")).setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_9fa6e3711337


In [40]:
val training_data = assembler.transform(data).select($"features")

training_data: org.apache.spark.sql.DataFrame = [features: vector]


In [41]:
val kmeans = new KMeans().setK(3).setSeed(1L)
val model = kmeans.fit(training_data)

kmeans: org.apache.spark.ml.clustering.KMeans = kmeans_4c4064542ae2
model: org.apache.spark.ml.clustering.KMeansModel = kmeans_4c4064542ae2


In [42]:
// Make predictions
val predictions = model.transform(training_data)
// Evaluate clustering by computing Silhouette score
val evaluator = new ClusteringEvaluator()
val silhouette = evaluator.evaluate(predictions)

predictions: org.apache.spark.sql.DataFrame = [features: vector, prediction: int]
evaluator: org.apache.spark.ml.evaluation.ClusteringEvaluator = cluEval_c1e834e01984
silhouette: Double = 0.6660332649991575


In [43]:
println(s"Silhouette with squared euclidean distance = $silhouette")
// Shows the result.
println("Cluster Centers: ")
model.clusterCenters.foreach(println)

Silhouette with squared euclidean distance = 0.6660332649991575
Cluster Centers: 
[7993.574780058651,4196.803519061584,5837.4926686217,2546.624633431085,2016.2873900293255,1151.4193548387098]
[9928.18918918919,21513.081081081084,30993.486486486487,2960.4324324324325,13996.594594594595,3772.3243243243246]
[35273.854838709674,5213.919354838709,5826.096774193548,6027.6612903225805,1006.9193548387096,2237.6290322580644]
